In [ ]:
import os
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.store.postgres import PostgresStore
from dotenv import load_dotenv
from utils import get_llm
from langmem import create_memory_manager, create_memory_store_manager
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [2]:
from pydantic import BaseModel

class PreferenceMemory(BaseModel):
    category: str
    preference: str
    context: str

In [3]:
model = get_llm()

In [4]:
manager = create_memory_manager(
    model,
    schemas=[PreferenceMemory],
    instructions="",
    enable_inserts=True
)

In [ ]:
# conversations_1 = [
#     HumanMessage("Alice likes dark mode in all of her apps"),
#     AIMessage("I now remember Alices preferences")
# ]

In [5]:
conversations_1 = [
            {"role": "user", "content": "Alice likes dark mode in all of her apps"},
            {"role": "assistant", "content": "I'll remember that preference"},
        ]

In [7]:
memories = await manager(conversations_1)

In [10]:
memories[0][1]

PreferenceMemory(category='appearance', preference='dark mode', context="Alice's apps")

In [11]:
conversations_2 = [
            {"role": "user", "content": "Alice lives in Hyderabad"},
            {"role": "assistant", "content": "I'll remember that preference"},
        ]

In [13]:
memories = await manager(conversations_2)


In [14]:
for memory in memories:
    print(memory[1])

category='person' preference='lives in Hyderabad' context='Alice'


In [21]:
conversations_3 = [
            {"role": "user", "content": "Bob lives in Hyderabad"},
            {"role": "user", "content": "Bob likes Biryani"},
            {"role": "assistant", "content": "I'll remember that preference"},
        ]

memories = await manager(conversations_3)

In [22]:
memories

[ExtractedMemory(id='42bda240-3c04-4821-9d11-eeb049d11e4f', content=PreferenceMemory(category='person', preference='lives in Hyderabad', context='Bob')),
 ExtractedMemory(id='86a353f9-86c4-4aeb-a72a-d29b4d544d6e', content=PreferenceMemory(category='person', preference='likes Biryani', context='Bob'))]